In [1]:
import torch
from torch.utils import data
import numpy as np
import h5py
import os
from time import time
import faiss
import io
import sys

In [4]:
dataset = "../dataset/deep10M"
max_points_per_cluster = 40
model_dict = torch.load(dataset+".pth")
ids = model_dict['ids']
cluster_ids = model_dict['cluster_idx']
index = model_dict['index']
centroids = model_dict['centroids']
print(index)


tensor([206094, 113489,  56939,  31248,  17655,   9625,   5733,   3154,   1674,
         43180])


In [3]:

current_dir = os.getcwd()
file_path = os.path.join(current_dir, "../dataset/deep-image-96-euclidean.hdf5")
deep10M = h5py.File(file_path, "r")
test_x = deep10M['test'][:]
train_x = deep10M['train'][:]
train_x = ((train_x + 1.0) * 127.5 + 0.5).astype(int)
test_x = ((test_x + 1.0) * 127.5 + 0.5).astype(int)
test_x = torch.from_numpy(test_x)
train_x = torch.from_numpy(train_x)
print("Dataset load done!")

Dataset load done!


In [ ]:
np.savetxt("./dataset/"+dataset+"_test.txt",test_x,fmt="%d",delimiter=" ")
np.savetxt("./dataset/"+dataset+"_dataset.txt",train_x,fmt="%d",delimiter=" ")
np.savetxt("./dataset/"+dataset+"_centroids.txt",centroids, fmt = "%d", delimiter= " ")

In [78]:
d=test_x.shape[1]
print("dim: ",d)
searchs = faiss.IndexFlatL2(d)
searchs.add(train_x)
D,res = searchs.search(test_x, 10)
neighbors = torch.from_numpy(res)
np.savetxt("./dataset/"+dataset+"_neighbors.txt",neighbors,fmt = "%d", delimiter= " ")

dim:  96


In [79]:
""" Maps cluster IDs to point IDs """
def save_ptoc(cluster_number, max_points_per_cluster, save):
    result = torch.empty((cluster_number, max_points_per_cluster))
    result.fill_(111111111)
    p_ids = ids.reshape(-1).type(torch.int)
    c_ids = cluster_ids.reshape(-1).type(torch.int)
    order = torch.argsort(c_ids)
    p_ids = p_ids[order]
    c_ids = c_ids[order] 
    counts = torch.bincount(c_ids)
    split_p_ids = torch.split(p_ids,counts.tolist(),dim=0)
    for i in range(cluster_number):
        cluster_points = split_p_ids[i]
        result[i, :len(cluster_points)] = cluster_points
    if save == True:
        np.savetxt("./dataset/"+dataset+"_ptoc.txt",result,fmt="%d",delimiter=" ")
    return result

In [80]:
all_cluster = centroids.shape[0]
num_cluters = sum(index) - index[-1]
stash_size = index[-1]
print("Total:",all_cluster)
print("Num cluster:", num_cluters," Stash size:", stash_size)
ptoc = save_ptoc(num_cluters,max_points_per_cluster,True)

Total: 488791
Num cluster: tensor(445611)  Stash size: tensor(43180)


In [81]:
# Save Stash
_, stash_id = searchs.search(centroids[num_cluters:],1)
stash_id = torch.from_numpy(stash_id)
np.savetxt("./dataset/"+dataset+"_stash.txt", stash_id, fmt="%d", delimiter= " ")

In [82]:
print("centroids:",centroids.shape)
print("stash:",stash_id.shape)
print("dataset:",train_x.shape)
print("test:",test_x.shape)
print("neighbors:",neighbors.shape)
print("ptoc:",ptoc.shape)

centroids: torch.Size([488791, 96])
stash: torch.Size([43180, 1])
dataset: torch.Size([9990000, 96])
test: torch.Size([10000, 96])
neighbors: torch.Size([10000, 10])
ptoc: torch.Size([445611, 40])
